In [1]:
from typing import Mapping, Iterator
import numpy as np
import os
from problog.logic import Term, Constant
from deepproblog.dataset import Dataset
from deepproblog.query import Query
import torch
import torchvision
import torchvision.transforms as transforms


In [2]:
class PuzzleDataset(Mapping[Term, torch.Tensor]):

    def __init__(self, subset: str, puzzle_path: str):
        """
        :param subset: Either 'train' or 'test'.
        :param puzzle_path: Path to the directory containing puzzle .npy files.
        :param label_path: Path to the directory containing label .npy files.
        """
        self.subset = subset
        self.puzzle_files = sorted(
            [f for f in os.listdir(puzzle_path) if f.endswith('.npy')]
        )
        self.puzzles = []
        for filename in self.puzzle_files:
            puzzle = np.load(os.path.join(puzzle_path, filename), allow_pickle=True)
            for i in range(len(puzzle)):
                for j in range(len(puzzle[i])):
                    if puzzle[i][j] is None:
                        puzzle[i][j] = np.zeros((28, 28), dtype=np.float32) / 255.0  # Create normalized zeros
                    else:
                        puzzle[i][j] = puzzle[i][j].astype(np.float32) / 255.0  # Normalize existing values
            self.puzzles.append(puzzle)

    def __len__(self) -> int:
        return len(self.puzzles)

    def __iter__(self) -> Iterator:
        for i in range(len(self)):
            yield self.puzzles[i]

    def __getitem__(self, item): #-> torch.Tensor:
        return torch.tensor(self.puzzles[int(item[0])][0][0]).unsqueeze(0)
        # return torch.from_numpy(self.puzzles[item])


In [3]:
class AdditionDataset(Dataset):

    def __init__(self, subset: str, labels_path: str):
        self.subset = subset
        self.label_files = sorted(
            [f for f in os.listdir(labels_path) if f.endswith('.npy')]
        )
        self.labels = []
        for filename in self.label_files:
            label = np.load(os.path.join(labels_path, filename), allow_pickle=True)
            for i in range(len(label)):
                for j in range(len(label[i])):
                    if label[i][j] is None:
                        label[i][j] = 0
            # label = label.astype(np.uint8)
            self.labels.append(label)

    def __len__(self):
        return len(self.labels) // 2

    def to_query(self, i: int) -> Query:
        image1_id = i*2
        image2_id = i*2+1
        image1_label = self.labels[image1_id].flatten()[0]
        image2_label = self.labels[image2_id].flatten()[0]
        image1 = Term("tensor", Term(self.subset, Constant(image1_id)))
        image2 = Term("tensor", Term(self.subset, Constant(image2_id)))
        label = Constant(int(image1_label) + int(image2_label))
        term = Term('addition', image1, image2, label)
        return Query(term)

ym = AdditionDataset('test', '../mnist_sudoku_generator/dataset/arrays/puzzles/test')
ym.to_query(0)

(1.0::addition(tensor(test(0)),tensor(test(1)),5), {})

In [4]:
import torch.nn as nn


class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.MaxPool2d(2, 2),  # 6 24 24 -> 6 12 12
            nn.ReLU(True),
            nn.Conv2d(6, 16, 5),  # 6 12 12 -> 16 8 8
            nn.MaxPool2d(2, 2),  # 16 8 8 -> 16 4 4
            nn.ReLU(True),
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
            nn.Softmax(1),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1, 16 * 4 * 4)
        x = self.classifier(x)
        return x


In [5]:
import torch

from deepproblog.dataset import DataLoader
from deepproblog.engines import ExactEngine
from deepproblog.model import Model
from deepproblog.network import Network
from deepproblog.train import train_model

network = MNIST_Net()
net = Network(network, "mnist_net", batching=True)
net.optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)

model = Model("addition.pl", [net])
model.set_engine(ExactEngine(model))
train = PuzzleDataset("train", "../mnist_sudoku_generator/dataset/images/puzzles/train")
test = PuzzleDataset("test", "../mnist_sudoku_generator/dataset/images/puzzles/test")
model.add_tensor_source("train", train)
model.add_tensor_source("test", test)

dataset = AdditionDataset('train', '../mnist_sudoku_generator/dataset/arrays/puzzles/train')


In [2]:
# Train the model
loader = DataLoader(dataset, 2, False)
train_model(model, loader, 1, log_iter=100, profile=0)
model.save_state("snapshot/trained_model.pth")

# Query the model
query = dataset.to_query(0)
result = model.solve([query])[0]
print(result)

NameError: name 'DataLoader' is not defined

In [15]:
dataset_test = AdditionDataset('test', '../mnist_sudoku_generator/dataset/arrays/puzzles/test')
query = dataset_test.to_query(5)
model.eval()
result = model.solve([query])[0]
print(result)

{addition(tensor(test(10)),tensor(test(11)),2): tensor(0.9118, grad_fn=<MulBackward0>)}


In [16]:
from deepproblog.evaluate import get_confusion_matrix
dataset_test = AdditionDataset('test', '../mnist_sudoku_generator/dataset/arrays/puzzles/test')
cm = get_confusion_matrix(model, dataset)


In [21]:
from deepproblog.evaluate import get_confusion_matrix
dataset_test = AdditionDataset('test', '../mnist_sudoku_generator/dataset/arrays/puzzles/test')
cm_test = get_confusion_matrix(model, dataset_test)


In [24]:
get_confusion_matrix(model, dataset_test)


In [22]:
cm_test.accuracy()

Accuracy:  0.912


np.float64(0.912)

In [17]:
cm.accuracy()

Accuracy:  0.90625


np.float64(0.90625)

In [20]:
str(cm)

'         \t  \t   \t   \t   \t   \t   \t   \t   \t   \tActual\t   \t  \t  \t  \t  \t  \t  \t  \t  \n         \t  \t  6\t  8\t  3\t  0\t  5\t  1\t  9\t  7\t     2\t  4\t11\t12\t10\t14\t13\t16\t15\t17\n         \t 6\t301\t  5\t  0\t  0\t  2\t  0\t  4\t  2\t     1\t  8\t 1\t 0\t 2\t 0\t 0\t 0\t 0\t 0\n         \t 8\t  3\t257\t 11\t  0\t 11\t  1\t  4\t  1\t     2\t  2\t 3\t 0\t 2\t 0\t 0\t 0\t 0\t 0\n         \t 3\t  0\t  8\t297\t  0\t  5\t  0\t  1\t  0\t     3\t  3\t 0\t 0\t 0\t 0\t 0\t 0\t 0\t 0\n         \t 0\t  0\t  0\t  0\t956\t  0\t  0\t  0\t  0\t     0\t  0\t 0\t 0\t 0\t 0\t 0\t 0\t 0\t 0\n         \t 5\t 14\t 20\t 12\t  0\t343\t  0\t  4\t  3\t     1\t  0\t 1\t 1\t 1\t 0\t 0\t 0\t 0\t 0\n         \t 1\t  3\t  1\t  0\t  0\t  1\t212\t  0\t  1\t     0\t  0\t 0\t 0\t 0\t 0\t 0\t 0\t 0\t 0\n         \t 9\t  1\t  3\t  0\t  0\t  4\t  0\t127\t  3\t     0\t  4\t 5\t 0\t 5\t 1\t 1\t 0\t 1\t 0\n         \t 7\t  5\t 10\t  9\t  0\t  3\t  1\t  7\t320\t     9\t  0\t 1\t 2\t 1\t 0\t 0\t 1\t 0\t 0\

In [23]:
print(str(cm_test))

         	  	  	  	  	   	  	  	  	  	Actual	  	  	  	  	  	  	  	  
         	  	 5	 6	 1	  0	 4	 2	 7	11	    13	10	 8	 9	 3	14	12	15	16
         	 5	90	 3	 0	  0	 0	 0	 0	 0	     0	 3	 7	 0	 2	 0	 0	 0	 0
         	 6	 2	76	 0	  0	 0	 1	 0	 0	     0	 0	 0	 0	 0	 0	 1	 0	 0
         	 1	 0	 1	53	  0	 0	 0	 0	 0	     0	 0	 1	 0	 1	 0	 0	 0	 0
         	 0	 0	 0	 0	239	 0	 0	 0	 0	     0	 0	 0	 0	 0	 0	 0	 0	 0
         	 4	 1	 0	 0	  0	66	 0	 1	 0	     0	 1	 0	 1	 0	 0	 0	 0	 0
         	 2	 0	 0	 0	  0	 0	68	 1	 0	     0	 0	 2	 0	 6	 0	 0	 0	 0
         	 7	 1	 0	 0	  0	 0	 0	70	 0	     0	 0	 1	 3	 3	 0	 0	 0	 0
Predicted	11	 0	 1	 0	  0	 0	 0	 1	24	     1	 0	 0	 1	 0	 0	 0	 0	 0
         	13	 0	 0	 0	  0	 0	 0	 0	 1	     9	 0	 1	 0	 0	 1	 0	 0	 0
         	10	 0	 1	 0	  0	 1	 0	 0	 2	     2	34	 0	 0	 0	 0	 1	 1	 0
         	 8	 2	 0	 0	  0	 0	 0	 0	 0	     2	 1	47	 4	 4	 0	 0	 0	 0
         	 9	 1	 1	 0	  0	 1	 0	 1	 0	     0	 2	 1	32	 0	 0	 0	 0	 0
         	 3	 2	 0	 0	  0	 0	 0	 0

In [144]:
from PIL import Image
Image.fromarray(test[[2]].numpy().reshape(28, 28) * 255).show()
Image.fromarray(test[[3]].numpy().reshape(28, 28) * 255).show()

In [108]:
from typing import Mapping, Iterator

import torch
import torchvision
import torchvision.transforms as transforms
from problog.logic import Term, Constant

from deepproblog.dataset import Dataset
from deepproblog.query import Query

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

datasets = {
    "train": torchvision.datasets.MNIST(
        root='data/', train=True, download=True, transform=transform
    ),
    "test": torchvision.datasets.MNIST(
        root='data/', train=False, download=True, transform=transform
    ),
}


class MNISTImages(Mapping[Term, torch.Tensor]):

    def __iter__(self) -> Iterator:
        for i in range(self.dataset):
            yield self.dataset[i][0]

    def __len__(self) -> int:
        return len(self.dataset)

    def __init__(self, subset):
        self.subset = subset
        self.dataset = datasets[self.subset]

    def __getitem__(self, item):
        print(self.dataset[int(item[0])][0])
        return self.dataset[int(item[0])][0]


class AdditionDataset(Dataset):

    def __init__(self, subset):
        self.subset = subset
        self.dataset = datasets[subset]

    def __len__(self):
        return len(self.dataset) // 2

    def to_query(self, i: int) -> Query:
        image1 = Term("tensor", Term(self.subset, Constant(i * 2)))
        image2 = Term("tensor", Term(self.subset, Constant(i * 2 + 1)))
        label = Constant(int(self.dataset[i*2][1] + self.dataset[i*2+1][1]))
        term = Term('addition', image1, image2, label)
        return Query(term)

ym = MNISTImages('train')
ym[[0]].shape

tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1

torch.Size([1, 28, 28])

In [29]:
dataset_test[1]

In [33]:
np.load('../mnist_sudoku_generator/dataset/arrays/puzzles/test/board_8000.npy', allow_pickle=True)

array([[5, 1, 8, 6, None, 3, 2, None, None],
       [None, 6, None, 5, 1, 2, None, 8, 7],
       [None, 3, None, None, 8, None, 1, None, 6],
       [3, 8, None, None, 6, None, None, 9, None],
       [None, None, 5, None, None, None, 6, 1, None],
       [None, 9, None, None, None, 5, None, 2, None],
       [6, None, None, None, 2, None, None, None, 4],
       [8, 7, 1, 9, None, 6, 5, None, 2],
       [9, 4, None, None, None, 7, 8, 6, None]], dtype=object)

In [35]:
img = np.load('../mnist_sudoku_generator/dataset/images/puzzles/test/board_8000.npy', allow_pickle=True)

In [41]:
img[0][0].shape

(28, 28)